<a href="https://colab.research.google.com/github/ssvadla/Research_01/blob/main/EM_03_Modular_UL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import f1_score



drive.mount('/content/drive')

train1 = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train2 = pd.read_csv('/content/drive/My Drive/Research/train_data2.csv')
train3 = pd.read_csv('/content/drive/My Drive/Research/train_data3.csv')
train4 = pd.read_csv('/content/drive/My Drive/Research/train_data4.csv')
train5 = pd.read_csv('/content/drive/My Drive/Research/train_data5.csv')
train6 = pd.read_csv('/content/drive/My Drive/Research/train_data6.csv')
train7 = pd.read_csv('/content/drive/My Drive/Research/train_data7.csv')
train8 = pd.read_csv('/content/drive/My Drive/Research/train_data8.csv')
train9 = pd.read_csv('/content/drive/My Drive/Research/train_data9.csv')
train10 = pd.read_csv('/content/drive/My Drive/Research/train_data10.csv')
train_highKappa = pd.read_csv('/content/drive/My Drive/Research/train_data_highkappa.csv')
train1.head()

train = train1
train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
for i in train_list:
  #print(i)
  train = train.append(i)


train.sort_values("Sentence", inplace = True)
print(len(train))

new_train = train.drop_duplicates(subset ="Sentence")

train = new_train
train['Target'].unique()
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()

#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text
len(train['Sentence'])
text = clean_text(train['Sentence'])
type(text)
len(text)


tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
print(X_tfidf.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
37711
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
(4416, 7374)


In [12]:
test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')

test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
t_p = tfidf_vect.transform(test['Sentence'])

In [13]:

unlabel = pd.read_csv(r'/content/drive/My Drive/Research/Unlabeled_data.csv')
#unlabel.head()

del unlabel['Complete']
del unlabel['Unnamed: 0']

unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))


def index_reset(unlabel_2):
  unlabel_2.reset_index(inplace=True)
  del unlabel_2['index']
  #print(unlabel_2.head())
  return unlabel_2


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
def feature_set_selection(train_and_unlabel_words_wo_duplicates,train_words,unlabel_words):
  for term in train_and_unlabel_words_wo_duplicates:
    #print(term)
    freq_train = train_words.count(term) / len(train_words)
    #print(freq_train)
    freq_unlabel = unlabel_words.count(term) / len(unlabel_words)
    #print(freq_unlabel)
    if freq_unlabel == 0:
      PF.append(term)
    elif (freq_train // freq_unlabel) > Threshold_feature:
      PF.append(term)
    else:
      NF.append(term)

In [15]:


def RN_selection(RN, unlabel_1):
  print(len(RN))
  #Q = pd.DataFrame( )
  iteration_RN = 0
  RN_to_be_removed = []
  pos_to_be_removed = []
  freq_each_word_list = []
  Q_pos = []

  count = 0
  if_count = 0

  for doc in unlabel_1['text']:
    #print("iteration_RN",iteration_RN)
    #print(doc)
    doc_words = clean_text(doc)
    doc_words_wo_duplicates = list(set(doc_words))
    for_count = 0
    
    for each_doc_word in doc_words_wo_duplicates:
      #print(each_doc_word)
      for_count = for_count + 1
      freq_each_doc_word = doc_words.count(each_doc_word)
      freq_each_word_list.append(freq_each_doc_word)
      #print(freq_each_doc_word)
      #print(each_doc_word in PF)
      if (freq_each_doc_word > 0) and (each_doc_word in PF):
        #print("Yes writing ........")
        if_count = if_count + 1
        pos_to_be_removed.append(count)
        RN_to_be_removed.append(doc)
        #print("breaking:::::::::::")
        break
      
    if for_count == len(doc_words_wo_duplicates):
      Q_pos.append(count)

    count = count + 1
    iteration_RN = iteration_RN + 1
  
  RN.drop(pos_to_be_removed,axis=0,inplace=True)
  #Q =  unlabel_1.loc[pos_to_be_removed,:]
  #print(len(Q))
  return RN,pos_to_be_removed, Q_pos, if_count



In [16]:


def classifier_select(train, Q, RN):
  f1_score_list = []
  per_CR_list = []

  loop_variable =0
  #for i in range(0,3):
  while(1):  # comment it change the range 
    print(loop_variable)
    p_and_RN = pd.concat([train,RN])
    print("#######target unique",np.unique(p_and_RN['Target']))
    p_and_RN.reset_index(inplace=True,drop=True)
    p_and_RN_vect = tfidf_vect.transform(p_and_RN['text'])
    p_and_RN_vect_df=pd.DataFrame(p_and_RN_vect.toarray())

    Q_vect = tfidf_vect.transform(Q['text'])
    Q_vect_df=pd.DataFrame(Q_vect.toarray())

    lgb_classifier = lgb.LGBMClassifier()
    lgb_classifier.fit(p_and_RN_vect_df, p_and_RN['Target'])
    np.unique(p_and_RN['Target'])

    #checking the classifier if it gives best results
    train_vect = tfidf_vect.transform(train['text'])
    train_vect_df=pd.DataFrame(train_vect.toarray())
    train_pred = lgb_classifier.predict(train_vect_df)
    print("#############train pred", np.unique(train_pred))
    classified_negative = (train_pred.tolist()).count(-1)
    print("unique predicted", np.unique(train_pred))
    print("classified_negative",classified_negative)
    print("len(train predic",len(train_pred))
    percentage = (classified_negative / len(train_pred)) * 100
    model_list.append(lgb_classifier)
    if percentage < percent_thresh:
      print("???????",percentage)
      print(percent_thresh)
      thresh_model_list.append(lgb_classifier)
      test_pred = lgb_classifier.predict(t_p.toarray())
      test['Target']= LabelEncoder().fit_transform(test['Target'])
      classification_report_test = classification_report(test['Target'],test_pred,digits=4)
      f1_score_test = f1_score(test['Target'],test_pred,average='weighted')
      f1_score_list.append(f1_score_test)
      per_CR_list.append(classification_report_test)


          

    Q_pred = lgb_classifier.predict(Q_vect_df)
    np.unique(Q_pred)

    

    count_q = 0
    total_q = 0 
    out_pos_q = []
    pos_q = []
    for i in Q_pred:
      #print(i)
      if i == -1:
        pos_q.append(count_q)
        total_q = total_q + 1
      else:
        out_pos_q.append(count_q)

      count_q = count_q + 1

    Q.reset_index(inplace=True,drop=True)
    W = Q.loc[pos_q,:]


    if W.empty :
      print("W is empty, came out of loop")
      break
    else:
      Q_new = Q.loc[out_pos_q,:]
      Q = Q_new.copy(deep =True)
      RN = pd.concat([RN,W])
      RN.reset_index(inplace=True,drop=True)
      loop_variable = loop_variable + 1
    print("completed iteration")
  
  result_dict['F1_score'].append(f1_score_list)
  result_dict['CR'].append(per_CR_list)
    

  return lgb_classifier, model_list , thresh_model_list

In [18]:
model_list = []
thresh_model_list = []
UL_size_list = []

result_dict = {"F1_score":[],"CR":[],"Unlabel_size":[]}
Threshold_feature = 1
percent_thresh = 5

In [19]:
# def EM_training(unlabel_1,percent_thresh,train):
#   print(train.columns)
#   PF = []
#   NF = []
#   unlabel_1['Target']=-1

#   train = train.rename(columns={'Sentence':'text'})
  
#   train['Target']= LabelEncoder().fit_transform(train['Target'])
#   train_and_unlabel =  pd.concat([train,unlabel_1])
#   print(len(train))
#   print(len(unlabel_1))
#   print(len(train_and_unlabel))
#   train = train.rename(columns={'Sentence':'text'})

#   train['Target']= LabelEncoder().fit_transform(train['Target'])
#   train_and_unlabel =  pd.concat([train,unlabel_1])
#   train_and_unlabel.reset_index(inplace=True) #commented now
#   del train_and_unlabel['index']
#   del train_and_unlabel['Unnamed: 0']
#   train_and_unlabel_words = clean_text(train_and_unlabel['text'])
#   train_and_unlabel_words_wo_duplicates = list(set(train_and_unlabel_words))
#   train_words = clean_text(train['text'])
#   train_words_wo_duplicates = list(set(train_words))
#   unlabel_words = clean_text(unlabel_1['text'])
#   unlabel_words_wo_duplicates = list(set(unlabel_words))

#   feature_set_selection(train_and_unlabel_words_wo_duplicates,train_words,unlabel_words)

#   RN = unlabel_1.copy(deep=True) 

#   RN,op_pos_to_be_removed, op_Q_pos, op_if_count = RN_selection(RN,unlabel_1) 

#   Q =  unlabel_1.loc[op_pos_to_be_removed,:]

  
#   lgb_classifier, op_model_list, op_thresh_model_list = classifier_select(train, Q, RN)


In [20]:
#unlabel_range = range(0,10000,5000)
unlabel_range = [500,1000]

In [21]:
#size = 500

print(train.columns)
for size in unlabel_range:
  unlabel_1 = unlabel.loc[:size]
  unlabel_1 = index_reset(unlabel_1)
  unlabel_1_copy = unlabel_1.copy(deep = True)
  x_un1 = tfidf_vect.transform(unlabel_1['text'])

  #EM_training(unlabel_1,percent_thresh,train)

  PF = []
  NF = []
  unlabel_1['Target']=-1

  train = train.rename(columns={'Sentence':'text'})
  
  train['Target']= LabelEncoder().fit_transform(train['Target'])
  train_and_unlabel =  pd.concat([train,unlabel_1])
  print(len(train))
  print(len(unlabel_1))
  print(len(train_and_unlabel))
  train = train.rename(columns={'Sentence':'text'})

  train['Target']= LabelEncoder().fit_transform(train['Target'])
  train_and_unlabel =  pd.concat([train,unlabel_1])
  train_and_unlabel.reset_index(inplace=True) #commented now
  del train_and_unlabel['index']
  del train_and_unlabel['Unnamed: 0']
  train_and_unlabel_words = clean_text(train_and_unlabel['text'])
  train_and_unlabel_words_wo_duplicates = list(set(train_and_unlabel_words))
  train_words = clean_text(train['text'])
  train_words_wo_duplicates = list(set(train_words))
  unlabel_words = clean_text(unlabel_1['text'])
  unlabel_words_wo_duplicates = list(set(unlabel_words))

  feature_set_selection(train_and_unlabel_words_wo_duplicates,train_words,unlabel_words)

  RN = unlabel_1.copy(deep=True) 

  RN,op_pos_to_be_removed, op_Q_pos, op_if_count = RN_selection(RN,unlabel_1) 

  Q =  unlabel_1.loc[op_pos_to_be_removed,:]

  result_dict['Unlabel_size'].append(size) 
  lgb_classifier, op_model_list, op_thresh_model_list = classifier_select(train, Q, RN)
  










    

Index(['Unnamed: 0', 'Sentence', 'Target'], dtype='object')
4416
501
4917


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


501
0
#######target unique [-1  0  1  2  3  4  5]
#############train pred [-1  0  1  2  3  4  5]
unique predicted [-1  0  1  2  3  4  5]
classified_negative 12
len(train predic 4416
??????? 0.2717391304347826
5


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


completed iteration
1
#######target unique [-1  0  1  2  3  4  5]
#############train pred [-1  0  1  2  3  4  5]
unique predicted [-1  0  1  2  3  4  5]
classified_negative 9
len(train predic 4416
??????? 0.20380434782608695
5
completed iteration
2
#######target unique [-1  0  1  2  3  4  5]


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


completed iteration
5
#######target unique [-1  0  1  2  3  4  5]
#############train pred [-1  0  1  2  3  4  5]
unique predicted [-1  0  1  2  3  4  5]
classified_negative 57
len(train predic 4416
??????? 1.2907608695652173
5
completed iteration
6
#######target unique [-1  0  1  2  3  4  5]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#############train pred [-1  0  1  2  3  4  5]
unique predicted [-1  0  1  2  3  4  5]
classified_negative 54
len(train predic 4416
??????? 1.2228260869565217
5
W is empty, came out of loop


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
with open(r'/content/drive/My Drive/EM/Result.txt', 'a') as writefile:
        writefile.write("\n")
        writefile.write(" EM Results ")
        writefile.write("\n")
        writefile.write(" final dict ")
        writefile.write("\n")
        writefile.write(str(result_dict))
        writefile.write("\n")
        

In [23]:


for model in op_model_list:
  test_pred = model.predict(t_p.toarray())
  test['Target']= LabelEncoder().fit_transform(test['Target'])
  classification_report_test = classification_report(test['Target'],test_pred,digits=4)
  print(classification_report_test)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMe

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.3846    0.2597    0.3101        77
           1     0.6957    0.6154    0.6531        26
           2     0.6771    0.8951    0.7710       267
           3     0.6122    0.3614    0.4545        83
           4     0.9000    0.2647    0.4091        34
           5     0.5926    0.4706    0.5246        34

    accuracy                         0.6334       521
   macro avg     0.5517    0.4096    0.4460       521
weighted avg     0.6335    0.6334    0.6069       521

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4151    0.2857    0.3385        77
           1     0.7273    0.6154    0.6667        26
           2     0.6800    0.8914    0.7715       267
           3     0.5490    0.3373    0.4179        83
           4     0.9000    0.2647    0.4091        34
           5     0.5556 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMe

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4615    0.3117    0.3721        77
           1     0.7083    0.6538    0.6800        26
           2     0.6888    0.8951    0.7785       267
           3     0.5490    0.3373    0.4179        83
           4     0.9091    0.2941    0.4444        34
           5     0.6000    0.4412    0.5085        34

    accuracy                         0.6392       521
   macro avg     0.5595    0.4190    0.4573       521
weighted avg     0.6425    0.6392    0.6167       521

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4118    0.2727    0.3281        77
           1     0.7273    0.6154    0.6667        26
           2     0.6801    0.8839    0.7687       267
           3     0.5490    0.3373    0.4179        83
           4     0.9000    0.2647    0.4091        34
           5     0.5714 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMe

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4255    0.2597    0.3226        77
           1     0.6800    0.6538    0.6667        26
           2     0.6791    0.8876    0.7695       267
           3     0.5094    0.3253    0.3971        83
           4     0.9000    0.2647    0.4091        34
           5     0.5714    0.4706    0.5161        34

    accuracy                         0.6257       521
   macro avg     0.5379    0.4088    0.4401       521
weighted avg     0.6220    0.6257    0.5989       521

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4255    0.2597    0.3226        77
           1     0.6957    0.6154    0.6531        26
           2     0.6799    0.8989    0.7742       267
           3     0.5625    0.3253    0.4122        83
           4     0.9091    0.2941    0.4444        34
           5     0.5926 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4468    0.2727    0.3387        77
           1     0.7619    0.6154    0.6809        26
           2     0.6706    0.8539    0.7512       267
           3     0.6047    0.3133    0.4127        83
           4     0.9167    0.3235    0.4783        34
           5     0.5200    0.3824    0.4407        34

    accuracy                         0.6046       521
   macro avg     0.5601    0.3945    0.4432       521
weighted avg     0.6378    0.6046    0.5947       521

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.5098    0.3377    0.4062        77
           1     0.7273    0.6154    0.6667        26
           2     0.6942    0.8502    0.7643       267
           3     0.6000    0.3253    0.4219        83
           4     0.8462    0.3235    0.4681        34
           5     0.5714 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
for model in op_thresh_model_list:
  test_pred = model.predict(t_p.toarray())
  test['Target']= LabelEncoder().fit_transform(test['Target'])
  classification_report_test = classification_report(test['Target'],test_pred,digits=4)
  print(classification_report_test)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.3846    0.2597    0.3101        77
           1     0.6957    0.6154    0.6531        26
           2     0.6771    0.8951    0.7710       267
           3     0.6122    0.3614    0.4545        83
           4     0.9000    0.2647    0.4091        34
           5     0.5926    0.4706    0.5246        34

    accuracy                         0.6334       521
   macro avg     0.5517    0.4096    0.4460       521
weighted avg     0.6335    0.6334    0.6069       521



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4151    0.2857    0.3385        77
           1     0.7273    0.6154    0.6667        26
           2     0.6800    0.8914    0.7715       267
           3     0.5490    0.3373    0.4179        83
           4     0.9000    0.2647    0.4091        34
           5     0.5556    0.4412    0.4918        34

    accuracy                         0.6296       521
   macro avg     0.5467    0.4051    0.4422       521
weighted avg     0.6286    0.6296    0.6040       521

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4583    0.2857    0.3520        77
           1     0.6800    0.6538    0.6667        26
           2     0.6868    0.8951    0.7772       267
           3     0.5400    0.3253    0.4060        83
           4     0.9091    0.2941    0.4444        34
           5     0.5714 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4255    0.2597    0.3226        77
           1     0.6800    0.6538    0.6667        26
           2     0.6791    0.8876    0.7695       267
           3     0.5094    0.3253    0.3971        83
           4     0.9000    0.2647    0.4091        34
           5     0.5714    0.4706    0.5161        34

    accuracy                         0.6257       521
   macro avg     0.5379    0.4088    0.4401       521
weighted avg     0.6220    0.6257    0.5989       521

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4255    0.2597    0.3226        77
           1     0.6957    0.6154    0.6531        26
           2     0.6799    0.8989    0.7742       267
           3     0.5625    0.3253    0.4122        83
           4     0.9091    0.2941    0.4444        34
           5     0.5926 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMe

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.5098    0.3377    0.4062        77
           1     0.7273    0.6154    0.6667        26
           2     0.6942    0.8502    0.7643       267
           3     0.6000    0.3253    0.4219        83
           4     0.8462    0.3235    0.4681        34
           5     0.5714    0.4706    0.5161        34

    accuracy                         0.6200       521
   macro avg     0.5641    0.4175    0.4633       521
weighted avg     0.6555    0.6200    0.6164       521

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4565    0.2727    0.3415        77
           1     0.7619    0.6154    0.6809        26
           2     0.6778    0.8352    0.7483       267
           3     0.5918    0.3494    0.4394        83
           4     0.7857    0.3235    0.4583        34
           5     0.6000 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
